# ID2222 Data Mining - Homework 3

Notebook by Beatrice Insalata, Laura Puccioni

This homework is based on the paper "TRIÈst: Counting Local and Global Triangles in Fully-Dynamic Streams with Fixed Memory Size". The paper presents a suite of one-pass streaming algorithms to compute unbiased, low-variance, high-quality approximations of the global and local number of triangles in a fully dynamic graphs represented as an adversarial stream of edge insertions and deletions.